In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import concatenate
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import models

In [3]:
data_student = "/content/drive/nuinvtnu 2021/Github_Data_Codes/CTM_Hien/"
data_teacher = "/content/drive/nuinvtnu 2021/Github_Data_Codes/CTM_Hien/"
path = "/content/drive/nuinvtnu 2021/Github_Data_Codes/CTM_Hien/"

In [4]:
def twoTupleDic3():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          for jj in AA_list_sort:
             AA_dict[i+j+jj] = numm
             numm += 1
    return AA_dict
def twoTupleDic2():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          AA_dict[i+j] = numm
          numm += 1
    return AA_dict

def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)
num_folds = 5
TIME_STEPS = 33
INPUT_SIZE = 300

In [5]:
# Load Traindata
import pandas as pd
file_train_student = "train_data_31_Arathailiana_1_1.csv"
df_train_student =pd.read_csv(data_student+file_train_student, delimiter= ',')

texts_train_student =[] #PTMsequend kmer
for i in df_train_student['Sequence']:
  temp = ProSentence(i,k)
  texts_train_student.append(temp)
df_train_student['k_mer'] =texts_train_student
train_sequences_student = []
for each in texts_train_student:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences_student.append(each_index_list)
# Tokenizer train data input Word2vec
data_token_student = []
for i in df_train_student['k_mer']:
   data_token_student.append(i.split())

MAX_SEQUENCE_LENGTH_student = len(data_token_student[1])
Xtrain_student = pad_sequences(train_sequences_student, maxlen=MAX_SEQUENCE_LENGTH_student)
ytrain_student = np.array(df_train_student['Label'])
ytrain_student = np.array(ytrain_student)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain_student = lb.fit_transform(ytrain_student)
ytrain_student = to_categorical(ytrain_student)
ytrain_student.shape


(3064, 2)

**Define model**

In [7]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results



In [8]:
checkpoint = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=3,
            verbose=1, mode='auto')
result_test =data_student +"Result_KD_BiLSTM.txt"


In [10]:
#  Cross validation Knowlege distillation model
kf = KFold(n_splits=num_folds, shuffle=True)

X_student = Xtrain_student
Y_student = ytrain_student
model_teacher = models.load_model(path+'teacher.h5')
fold_idx =1
for train_index_student, test_index_student in kf.split(X_student,Y_student):
    X_train_student, X_test_student =X_student[train_index_student], X_student[test_index_student]
    Y_train_student, Y_test_student = Y_student[train_index_student], Y_student[test_index_student]
# Create the student
    print("Start train Fold ", fold_idx)
    student = keras.Sequential(
    [
        layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH_student,trainable=True),
        #layers.Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH_student,trainable=True),
        layers.Bidirectional(LSTM(units=16,batch_input_shape=(None,TIME_STEPS, INPUT_SIZE),return_sequences=True)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Dense(2),
        layers.Activation('softmax'),
    ],
    name="student",
    )
    student_scratch = keras.models.clone_model(student)
    distiller = Distiller(student=student, teacher= model_teacher)
    distiller.compile(
          student_loss_fn= keras.losses.CategoricalCrossentropy(from_logits=True),
          #student_loss_fn= keras.losses.BinaryCrossentropy(from_logits=True),
          optimizer=keras.optimizers.Adam(learning_rate = 0.0001),
          metrics=[
                  "categorical_accuracy",
                  "AUC",
                  ], # , f1
          distillation_loss_fn=keras.losses.KLDivergence(),
          alpha=0.1,
          temperature=10
    )
    history_KD = distiller.fit(X_train_student, Y_train_student, batch_size=16, validation_data =(X_test_student, Y_test_student), epochs=100, callbacks=[checkpoint],verbose=1)
    result2 = distiller.evaluate(X_test_student, Y_test_student)#, validation_data=(X_test,Y_test), shuffle = True,callbacks=[checkpoint],verbose=1)
    f = open(result_test, 'a+', encoding='UTF-8')
    f.write("\n KD_BiLSTM fold: \n")
    f.write(str(fold_idx))
    s = str(result2)
    f.write(s)
    f.close()


    ypred = student.predict(X_test_student)
    ypred =np.argmax(ypred,axis =1)

    ytest_true = np.argmax(Y_test_student,axis =1)

    result22 = confusion_matrix(ytest_true,ypred)
    print(result22)
    f = open(result_test, 'a+', encoding='UTF-8')
    f.write("\n Fold Confusion_matric: \n ")
    s = str(result22)
    f.write(s)
    f.close()
    fold_idx = fold_idx +1

Start train Fold  1
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


152/154 [============================>.] - ETA: 0s - categorical_accuracy: 0.6846 - auc: 0.7386 - student_loss: 0.6396 - distillation_loss: 0.0463

154/154 [==============================] - 10s 25ms/step - categorical_accuracy: 0.6830 - auc: 0.7362 - student_loss: 0.6384 - distillation_loss: 0.0459 - val_categorical_accuracy: 0.7488 - val_auc: 0.8201 - val_student_loss: 0.8870
Epoch 2/100
152/154 [============================>.] - ETA: 0s - categorical_accuracy: 0.7755 - auc: 0.8351 - student_loss: 0.5101 - distillation_loss: 0.0266

154/154 [==============================] - 3s 20ms/step - categorical_accuracy: 0.7756 - auc: 0.8353 - student_loss: 0.5087 - distillation_loss: 0.0263 - val_categorical_accuracy: 0.7569 - val_auc: 0.8493 - val_student_loss: 0.8758
Epoch 3/100
152/154 [============================>.] - ETA: 0s - categorical_accuracy: 0.7903 - auc: 0.8574 - student_loss: 0.4750 - distillation_loss: 0.0243

...

154/154 [==============================] - 3s 20ms/step - categorical_accuracy: 0.9168 - auc: 0.9794 - student_loss: 0.2162 - distillation_loss: 0.0091 - val_categorical_accuracy: 0.8611 - val_auc: 0.9299 - val_student_loss: 0.2276
Epoch 97/100
151/154 [============================>.] - ETA: 0s - categorical_accuracy: 0.9123 - auc: 0.9781 - student_loss: 0.2188 - distillation_loss: 0.0090

154/154 [==============================] - 3s 19ms/step - categorical_accuracy: 0.9127 - auc: 0.9781 - student_loss: 0.2199 - distillation_loss: 0.0090 - val_categorical_accuracy: 0.8529 - val_auc: 0.9291 - val_student_loss: 0.2022
Epoch 98/100
153/154 [============================>.] - ETA: 0s - categorical_accuracy: 0.9097 - auc: 0.9769 - student_loss: 0.2218 - distillation_loss: 0.0087

154/154 [==============================] - 3s 20ms/step - categorical_accuracy: 0.9095 - auc: 0.9769 - student_loss: 0.2238 - distillation_loss: 0.0089 - val_categorical_accuracy: 0.8529 - val_auc: 0.9286 - val_student_loss: 0.2078
Epoch 99/100
154/154 [==============================] - ETA: 0s - categorical_accuracy: 0.9172 - auc: 0.9777 - student_loss: 0.2217 - distillation_loss: 0.0090

154/154 [==============================] - 3s 21ms/step - categorical_accuracy: 0.9172 - auc: 0.9777 - student_loss: 0.2232 - distillation_loss: 0.0090 - val_categorical_accuracy: 0.8676 - val_auc: 0.9301 - val_student_loss: 0.2299
Epoch 100/100
154/154 [==============================] - ETA: 0s - categorical_accuracy: 0.9188 - auc: 0.9778 - student_loss: 0.2190 - distillation_loss: 0.0089

20/20 [==============================] - 1s 7ms/step
[[240  53]
 [ 31 288]]
